<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStardust/blob/main/_AdvancedNet_Residual_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install optuna

In [ ]:
pip install diffprivlib

In [ ]:
pip install aif360

In [ ]:
!pip install reco-gym
!pip install 'aif360[Reductions]'

In [ ]:

# Import Libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import logging
import requests
import optuna
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from scipy import stats
from albumentations import Compose, RandomRotate90, Flip, Transpose, RandomBrightnessContrast
from albumentations.pytorch import ToTensorV2
from diffprivlib.models import LogisticRegression as DPLogisticRegression
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets import BinaryLabelDataset
from reco_gym import build_agent_init, test_agent, env_1_args
from reco_gym.agent import random_agent_args, critical_agent_args
from torch.utils.tensorboard import SummaryWriter
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor, ConsoleSpanExporter

# Define AdvancedNet with Residual Connections
class AdvancedNet(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super(AdvancedNet, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc_residual = nn.Linear(784, 128)
        self.fc_out = nn.Linear(128, 10)

    def forward(self, x):
        x1 = torch.relu(self.bn1(self.fc1(x)))
        x2 = self.dropout(torch.relu(self.bn2(self.fc2(x1))))
        x_residual = self.fc_residual(x)
        x2 += x_residual
        output = self.fc_out(x2)
        return output

# Data Setup
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)

# Model, Optimizer, Criterion
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AdvancedNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Training Loop with Mixed Precision and Gradient Accumulation
scaler = torch.cuda.amp.GradScaler()
num_epochs = 10
accumulation_steps = 4
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.view(-1, 784).to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels) / accumulation_steps
        scaler.scale(loss).backward()
        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        running_loss += loss.item()
    scheduler.step()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader):.4f}")

# Save Model Checkpoints
def save_checkpoint(model, optimizer, epoch, version="1.0"):
    checkpoint_dir = f"checkpoints/v{version}"
    os.makedirs(checkpoint_dir, exist_ok=True)
    checkpoint_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch}.pth")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_path)
    print(f"Checkpoint saved: {checkpoint_path}")

# Model Evaluation with Metrics
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.view(-1, 784).to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    conf_matrix = confusion_matrix(all_labels, all_preds)
    print(f"F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
    print(f"Confusion Matrix:\n{conf_matrix}")

# Hyperparameter Tuning with Optuna
def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    model = AdvancedNet(dropout_rate=dropout_rate).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    val_loss = evaluate_model(model, DataLoader(datasets.MNIST(root='./data', train=False, download=True, transform=transform)))
    return val_loss

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)
print("Best Hyperparameters:", study.best_params)

# Logging
writer = SummaryWriter(log_dir='runs/experiment_1')
for epoch in range(num_epochs):
    writer.add_scalar('Loss/train', running_loss / len(train_loader), epoch)
writer.close()

# Slack Notification
def send_slack_notification(message, webhook_url):
    payload = {"text": message}
    response = requests.post(webhook_url, json=payload)
    return response.status_code

webhook_url = "YOUR_SLACK_WEBHOOK_URL"
message = "Drift detected in the model. Immediate action required."
send_slack_notification(message, webhook_url)

# Configure logging
logging.basicConfig(level=logging.INFO, filename='model_logs.log', filemode='a', format='%(asctime)s - %(levelname)s - %(message)s')

def log_model_event(event):
    logging.info(event)

log_model_event("Model training started.")
log_model_event("Model training completed.")

# Tracing setup
trace.set_tracer_provider(TracerProvider())
span_processor = BatchSpanProcessor(ConsoleSpanExporter())
trace.get_tracer_provider().add_span_processor(span_processor)

tracer = trace.get_tracer(__name__)

with tracer.start_as_current_span("main"):
    with tracer.start_as_current_span("data_loading"):
        # Data loading code
        pass

    with tracer.start_as_current_span("model_inference"):
        # Model inference code
        pass

# Load data
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

# Train base models
rf = RandomForestClassifier()
gb = GradientBoostingClassifier()
rf.fit(X_train, y_train)
gb.fit(X_train, y_train)

# Stack base model predictions
meta_features = np.column_stack((rf.predict(X_test), gb.predict(X_test)))

# Train meta-model
meta_model = LogisticRegression()
meta_model.fit(meta_features, y_test)

# Evaluate stacked model
meta_predictions = meta_model.predict(meta_features)
print(f"Stacked model accuracy: {accuracy_score(y_test, meta_predictions)}")

# Data Augmentation
augmentor = Compose([
    RandomRotate90(),
    Flip(),
    Transpose(),
    RandomBrightnessContrast(),
    ToTensorV2()
])

def augment_image(image):
    augmented = augmentor(image=image)
    return augmented['image']

# Example usage of augmented image
image = ...  # Replace with actual image data
augmented_image = augment_image(image)

# Differential Privacy
dp_model = DPLogisticRegression(epsilon=1.0)
dp_model.fit(X_train, y_train)
predictions = dp_model.predict(X_test)

# Fairness Metrics
df = ...  # Your dataframe here
dataset = BinaryLabelDataset(df=df, label_names=['label'], protected_attribute_names=['gender'])
metric = BinaryLabelDatasetMetric(dataset, privileged_groups=[{'gender': 1}], unprivileged_groups=[{'gender': 0}])
print("Disparate Impact:", metric.disparate_impact())

# Recommendation Agents
env = build_agent_init(env_1_args)
random_agent = build_agent_init(random_agent_args)
critical_agent = build_agent_init(critical_agent_args)

test_agent(env, random_agent)
test_agent(env, critical_agent)

In [ ]:
# Import Libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import logging
import requests
import optuna
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from torch.utils.tensorboard import SummaryWriter
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from scipy import stats
from albumentations import Compose, RandomRotate90, Flip, Transpose, RandomBrightnessContrast
from albumentations.pytorch import ToTensorV2
from diffprivlib.models import LogisticRegression as DPLogisticRegression
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets import BinaryLabelDataset

# Define AdvancedNet with Residual Connections
class AdvancedNet(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super(AdvancedNet, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc_residual = nn.Linear(784, 128)
        self.fc_out = nn.Linear(128, 10)

    def forward(self, x):
        x1 = torch.relu(self.bn1(self.fc1(x)))
        x2 = self.dropout(torch.relu(self.bn2(self.fc2(x1))))
        x_residual = self.fc_residual(x)
        x2 += x_residual
        output = self.fc_out(x2)
        return output

# Data Setup
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)

# Model, Optimizer, Criterion
model = AdvancedNet().to("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Training Loop with Mixed Precision and Gradient Accumulation
scaler = torch.cuda.amp.GradScaler()
num_epochs = 10
accumulation_steps = 4
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.view(-1, 784).to("cuda" if torch.cuda.is_available() else "cpu")
        labels = labels.to("cuda" if torch.cuda.is_available() else "cpu")
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels) / accumulation_steps
        scaler.scale(loss).backward()
        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        running_loss += loss.item()
    scheduler.step()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader):.4f}")

# Save Model Checkpoints
def save_checkpoint(model, optimizer, epoch, version="1.0"):
    checkpoint_dir = f"checkpoints/v{version}"
    os.makedirs(checkpoint_dir, exist_ok=True)
    checkpoint_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch}.pth")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_path)
    print(f"Checkpoint saved: {checkpoint_path}")

# Model Evaluation with Metrics
def evaluate_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.view(-1, 784).to("cuda" if torch.cuda.is_available() else "cpu")
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    conf_matrix = confusion_matrix(all_labels, all_preds)
    print(f"F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
    print(f"Confusion Matrix:\n{conf_matrix}")

# Logging and Notifications
def send_slack_notification(message, webhook_url):
    payload = {"text": message}
    response = requests.post(webhook_url, json=payload)
    return response.status_code

webhook_url = "YOUR_SLACK_WEBHOOK_URL"  # Replace with your actual Slack webhook URL
message = "Drift detected in the model. Immediate action required."
send_slack_notification(message, webhook_url)

# Configure logging
logging.basicConfig(level=logging.INFO, filename='model_logs.log', filemode='a',
                    format='%(asctime)s - %(levelname)s - %(message)s')

def log_model_event(event):
    logging.info(event)

log_model_event("Model training started.")
log_model_event("Model training completed.")

# Alternative Recommendation System Example with Ensemble Stacking
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

# Train base models
rf = RandomForestClassifier()
gb = GradientBoostingClassifier()
rf.fit(X_train, y_train)
gb.fit(X_train, y_train)

# Stack base model predictions
meta_features = np.column_stack((rf.predict(X_test), gb.predict(X_test)))

# Train meta-model
meta_model = LogisticRegression()
meta_model.fit(meta_features, y_test)

# Evaluate stacked model
meta_predictions = meta_model.predict(meta_features)
print(f"Stacked model accuracy: {accuracy_score(y_test, meta_predictions)}")

# Image Augmentation Setup
augmentor = Compose([
    RandomRotate90(),
    Flip(),
    Transpose(),
    RandomBrightnessContrast(),
    ToTensorV2()
])

def augment_image(image):
    augmented = augmentor(image=image)
    return augmented['image']

# Example usage
# image = ...  # Load or generate your image data here
# augmented_image = augment_image(image)

# Differentially Private Logistic Regression
dp_model = DPLogisticRegression(epsilon=1.0)
dp_model.fit(X_train, y_train)
dp_predictions = dp_model.predict(X_test)

# Fairness Metric Calculation with AIF360
# Assume you have a DataFrame for BinaryLabelDataset setup
# df = ...  # Load your DataFrame here
# dataset = BinaryLabelDataset(df=df, label_names=['label'], protected_attribute_names=['gender'])
# metric = BinaryLabelDatasetMetric(dataset, privileged_groups=[{'gender': 1}], unprivileged_groups=[{'gender': 0}])
# print("Disparate Impact:", metric.disparate_impact())

# Example of downloading MNIST dataset (already included in DataLoader setup)
# This part is commented out as the dataset is already being downloaded in the DataLoader setup.
# datasets.MNIST(root='./data', train=True, download=True)

# Final logging
log_model_event("Model evaluation completed.")